In [1]:
import pandas as pd
import os
import numpy as np
import statsmodels.api as sm
from sklearn import metrics

In [2]:
cwd = os.getcwd()
current_folder = cwd.split('\\')[-1]
print(current_folder)
if current_folder != 'BFOR-516-Project':
    os.chdir('..')
    cwd = os.getcwd()
current_folder = cwd.split('\\')[-1]
print(current_folder)
cwd

models
BFOR-516-Project


'C:\\Users\\jacks\\Documents\\GitHub\\BFOR-516-Project'

In [3]:
data = pd.read_csv('data/datafm20.csv')

In [4]:
data

,Name,Position,Club,Division,Based,Nation,Height,Weight,Age,Preferred Foot,...,Cmp,Com,Cmd,Bra,Bal,Ant,Agi,Agg,Aer,Acc
0,Lionel Messi,"AM (RC), ST (C)",Barcelona,Spanish First Division,Spain (First Division),ARG,170,72,32,Left,...,18,2,2,10,19,19,19,7,2,18
1,Cristiano Ronaldo,"AM (RL), ST (C)",Juventus,Italian Serie A,Italy (Serie A),POR,185,83,34,Either,...,14,4,1,16,14,18,13,6,2,15
2,Kylian MbappÃ©,"AM (RL), ST (C)",Paris SG,Ligue 1 Conforama,France (Ligue 1 Conforama),FRA,178,73,20,Right,...,18,2,1,12,14,17,16,6,4,20
3,Manuel Neuer,GK,FC Bayern,Bundesliga,Germany (Bundesliga),GER,192,90,33,Either,...,20,12,17,14,14,19,15,8,14,13
4,Neymar,"M (L), AM (LC), ST (C)",Paris SG,Ligue 1 Conforama,France (Ligue 1 Conforama),BRA,175,68,27,Right,...,17,3,1,14,14,16,18,10,4,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144745,Yuan Jinxing,ST (C),Unknown,Unknown,China,CHN,169,81,41,Right Only,...,7,3,3,2,3,9,8,8,1,11
144746,Aizudin Abdul Rahman,M (C),BSRC,Bruneian Premier League,Brunei (Premier League),BRU,180,69,24,Right Only,...,4,1,3,1,3,4,9,14,2,9
144747,Berlin Jean-Gilles,ST (C),CS Monteuil,PremiÃ¨re Ligue de soccer du QuÃ©bec,Canada (PLSQ),HAI,180,79,32,Right Only,...,8,3,1,1,2,4,10,6,3,11
144748,Frederik van Hauen,M (C),Skovshoved,2. Division,Denmark (2. Division),DEN,195,85,19,Right Only,...,3,3,3,3,4,4,3,16,1,5


In [5]:
data['log_wage'] = np.log(data['Wage']+1)
data['log_value'] = np.log(data['Value']+1)
data = pd.concat([data, pd.get_dummies(data['Nation'])], axis=1)
data = pd.concat([data, pd.get_dummies(data['Preferred Foot'])], axis=1)
data = pd.concat([data, pd.get_dummies(data['Best Pos'])], axis=1)
data = pd.concat([data, pd.get_dummies(data['Best Role'])], axis=1)
#data['Nation_dummy'] =pd.Categorical(data['Nation']).codes
#data['preferred_Foot_dummy'] =pd.Categorical(data['Preferred Foot']).codes
#data['Best_Pos_dummy'] =pd.Categorical(data['Best Pos']).codes
#data['Best_Role_dummy'] =pd.Categorical(data['Best Role']).codes

In [6]:
data

,Name,Position,Club,Division,Based,Nation,Height,Weight,Age,Preferred Foot,...,SS,T,TM,Unknown,VOL,W,WB,WM,WP,WTM
0,Lionel Messi,"AM (RC), ST (C)",Barcelona,Spanish First Division,Spain (First Division),ARG,170,72,32,Left,...,0,0,0,0,0,0,0,0,0,0
1,Cristiano Ronaldo,"AM (RL), ST (C)",Juventus,Italian Serie A,Italy (Serie A),POR,185,83,34,Either,...,0,0,0,0,0,0,0,0,0,0
2,Kylian MbappÃ©,"AM (RL), ST (C)",Paris SG,Ligue 1 Conforama,France (Ligue 1 Conforama),FRA,178,73,20,Right,...,0,0,0,0,0,0,0,0,0,0
3,Manuel Neuer,GK,FC Bayern,Bundesliga,Germany (Bundesliga),GER,192,90,33,Either,...,0,0,0,0,0,0,0,0,0,0
4,Neymar,"M (L), AM (LC), ST (C)",Paris SG,Ligue 1 Conforama,France (Ligue 1 Conforama),BRA,175,68,27,Right,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144745,Yuan Jinxing,ST (C),Unknown,Unknown,China,CHN,169,81,41,Right Only,...,0,0,0,0,0,0,0,0,0,0
144746,Aizudin Abdul Rahman,M (C),BSRC,Bruneian Premier League,Brunei (Premier League),BRU,180,69,24,Right Only,...,0,0,0,0,0,0,0,0,0,0
144747,Berlin Jean-Gilles,ST (C),CS Monteuil,PremiÃ¨re Ligue de soccer du QuÃ©bec,Canada (PLSQ),HAI,180,79,32,Right Only,...,0,0,0,0,0,0,0,0,0,0
144748,Frederik van Hauen,M (C),Skovshoved,2. Division,Denmark (2. Division),DEN,195,85,19,Right Only,...,0,0,0,0,0,0,0,0,0,0


In [7]:
minimums = data.min()
maximums = data.max()
#minimums.to_csv('minimums.csv')
#maximums.to_csv('maximums.csv')

Value and wage have 0s so might have to use log for those two

In [8]:
data.dtypes

Name        object
Position    object
Club        object
Division    object
Based       object
             ...  
W            uint8
WB           uint8
WM           uint8
WP           uint8
WTM          uint8
Length: 344, dtype: object

In [10]:
#data['Nation_dummy'].max()

In [11]:
int_list = []
pred_vars = []

for i in data.columns: 
    #print(i, ' ', data[i].dtypes)
    if 'int' in str(data[i].dtypes) or 'float' in str(data[i].dtypes):
        if 'uint' not in str(data[i].dtypes):
            int_list.append(i)
        if i != 'CA' and i !='PA' and i != 'Wage' and i != 'Value':
            pred_vars.append(i)

In [12]:
len(pred_vars)

331

In [13]:
corr = data[int_list].corr()
corr.style.background_gradient(cmap='coolwarm')

,Height,Weight,Age,Value,Wage,CA,PA,Wor,Vis,Thr,Tec,Tea,Tck,Str,Sta,TRO,Ref,Pun,Pos,Pen,Pas,Pac,1v1,OtB,Nat,Mar,L Th,Lon,Ldr,Kic,Jum,Hea,Han,Fre,Fla,Fir,Fin,Ecc,Dri,Det,Dec,Cro,Cor,Cnt,Cmp,Com,Cmd,Bra,Bal,Ant,Agi,Agg,Aer,Acc,log_wage,log_value
Height,1.000000,0.790270,0.048629,0.021120,0.024942,0.076890,0.059560,-0.141420,-0.192301,0.272584,-0.292641,-0.008698,-0.019366,0.360370,-0.105444,0.282027,0.303359,0.279315,0.229945,-0.067279,-0.137968,-0.232315,0.282575,-0.241078,0.017983,0.025868,-0.070726,-0.215612,0.054739,0.288263,0.769151,0.139777,0.303184,-0.074850,-0.236316,-0.297125,-0.178392,0.162173,-0.288349,0.020050,0.084630,-0.298818,-0.240485,0.127001,-0.028173,0.283253,0.290370,0.235186,0.092150,0.089000,-0.243586,0.032998,0.327978,-0.262820,0.035050,0.032722
Weight,0.790270,1.000000,0.274063,0.049281,0.058936,0.249639,0.161103,0.008743,-0.063505,0.249706,-0.158673,0.165080,0.051282,0.551687,0.062155,0.233436,0.250476,0.232060,0.326303,0.087870,-0.021370,-0.234372,0.254992,-0.092786,-0.041850,0.106106,0.063888,-0.071271,0.141025,0.248535,0.694555,0.231907,0.249714,0.056031,-0.138915,-0.191685,-0.069524,0.158504,-0.165969,0.053931,0.097234,-0.164049,-0.107743,0.277984,0.138192,0.254347,0.253600,0.348554,0.282753,0.256208,-0.215178,0.073557,0.275978,-0.271931,0.086964,0.058074
Age,0.048629,0.274063,1.000000,0.040546,0.073089,0.296342,0.092173,0.235134,0.173740,0.050242,0.127561,0.330482,0.086274,0.379263,0.315064,0.018469,0.006307,0.019915,0.243485,0.309101,0.156565,-0.145307,0.048038,0.196127,-0.004369,0.123241,0.232615,0.190034,0.303110,0.023570,0.151741,0.156826,0.009398,0.241828,0.145065,0.055901,0.115485,0.057289,0.107136,0.084557,0.012726,0.168790,0.172092,0.340873,0.310602,0.050252,0.033244,0.286951,0.335420,0.350986,-0.064017,0.107875,0.028374,-0.195395,0.027583,-0.090926
Value,0.021120,0.049281,0.040546,1.000000,0.763292,0.326957,0.286218,0.178467,0.177202,0.004797,0.169782,0.208125,0.078166,0.166927,0.177811,-0.003040,-0.005287,-0.008685,0.100763,0.183799,0.176875,0.128165,0.005524,0.154641,0.076975,0.066055,0.104486,0.155860,0.069209,-0.001638,0.082608,0.097222,-0.008642,0.156182,0.136361,0.145076,0.129317,0.008712,0.151187,0.097741,0.105903,0.142231,0.133609,0.188301,0.215416,0.001132,-0.001887,0.163199,0.212499,0.205335,0.130989,0.071993,-0.005238,0.101920,0.270435,0.243476
Wage,0.024942,0.058936,0.073089,0.763292,1.000000,0.306466,0.270332,0.161506,0.164798,0.010082,0.158015,0.199219,0.071789,0.164677,0.164583,0.001683,0.000324,-0.004923,0.098836,0.180503,0.165641,0.107111,0.012312,0.142671,0.068988,0.060405,0.105603,0.147875,0.075424,0.002603,0.084804,0.097449,-0.002833,0.153742,0.126871,0.133886,0.121339,0.015235,0.136399,0.092806,0.099960,0.131787,0.127897,0.181935,0.207909,0.007815,0.004356,0.159950,0.206039,0.200642,0.113288,0.066046,0.000205,0.077994,0.282082,0.218056
CA,0.076890,0.249639,0.296342,0.326957,0.306466,1.000000,0.859521,0.581409,0.561502,0.073998,0.531869,0.709459,0.298931,0.623386,0.567241,-0.009060,0.001791,-0.012333,0.475408,0.543884,0.601253,0.269658,0.060882,0.492931,0.079361,0.317486,0.479528,0.521084,0.223545,0.030932,0.330086,0.395560,-0.002178,0.560957,0.349731,0.430831,0.379396,0.048916,0.458322,0.184661,0.290213,0.486116,0.524601,0.663173,0.662044,0.045140,0.029239,0.555406,0.723519,0.687894,0.302087,0.140309,0.010254,0.188974,0.574260,0.552499
PA,0.059560,0.161103,0.092173,0.286218,0.270332,0.859521,1.000000,0.485905,0.509560,0.067196,0.490375,0.602679,0.264767,0.509762,0.433355,-0.004851,0.006144,-0.009872,0.390118,0.469681,0.552309,0.215659,0.057120,0.414957,0.070558,0.261362,0.398674,0.464093,0.141624,0.032341,0.277155,0.345306,0.002223,0.492098,0.317330,0.401876,0.339272,0.040638,0.413734,0.175693,0.264140,0.424214,0.462811,0.548684,0.555864,0.036999,0.026156,0.488480,0.610402,0.573239,0.241473,0.115911,0.008599,0.150812,0.542212,0.543839
Wor,-0.141420,0.008743,0.235134,0.178467,0.161506,0.581409,0.485905,1.000000,0.603221,-0.180702,0.574603,0.749532,0.399150,0.440133,0.636

In [14]:
corr2 = corr[['CA', 'PA']]
corr2.style.background_gradient(cmap='coolwarm')

,CA,PA
Height,0.076890,0.059560
Weight,0.249639,0.161103
Age,0.296342,0.092173
Value,0.326957,0.286218
Wage,0.306466,0.270332
CA,1.000000,0.859521
PA,0.859521,1.000000
Wor,0.581409,0.485905
Vis,0.561502,0.509560
Thr,0.073998,0.067196


In [15]:
from sklearn.model_selection import train_test_split

# use index-based sampling since we have time series data
train, test = train_test_split(data, test_size=0.25, shuffle=True)

In [16]:
print(len(train))
# train = train.dropna(subset=pred_vars)
print(len(test))

108562
36188


In [17]:
from sklearn.ensemble import RandomForestRegressor

rf_reg_PA = RandomForestRegressor()

rf_reg_PA.fit(train[pred_vars], train['PA'])

rf_reg_CA = RandomForestRegressor()

rf_reg_CA.fit(train[pred_vars], train['CA'])

RandomForestRegressor()

In [18]:
#About an hour and half
#Took forever and RMSE was average
'''from sklearn.svm import SVR

svr_reg_PA = SVR()

svr_reg_PA.fit(train[pred_vars], train['PA'])

svr_reg_CA = SVR()

svr_reg_CA.fit(train[pred_vars], train['CA'])'''

"from sklearn.svm import SVR\n\nsvr_reg_PA = SVR()\n\nsvr_reg_PA.fit(train[pred_vars], train['PA'])\n\nsvr_reg_CA = SVR()\n\nsvr_reg_CA.fit(train[pred_vars], train['CA'])"

In [19]:
#MLP preformed horribly, not horrible anymore
from sklearn.neural_network import MLPRegressor

mlp_reg_CA = MLPRegressor(max_iter=220, verbose=True)

mlp_reg_CA.fit(train[pred_vars], train['CA'])

mlp_reg_PA = MLPRegressor(max_iter=220, verbose=True)

mlp_reg_PA.fit(train[pred_vars], train['PA'])

Iteration 1, loss = 130.18399287
Iteration 2, loss = 23.17601436
Iteration 3, loss = 21.15975464
Iteration 4, loss = 20.48733236
Iteration 5, loss = 19.99006087
Iteration 6, loss = 19.38509615
Iteration 7, loss = 18.41704012
Iteration 8, loss = 17.18023312
Iteration 9, loss = 16.21178620
Iteration 10, loss = 15.48427697
Iteration 11, loss = 15.09229020
Iteration 12, loss = 14.67385772
Iteration 13, loss = 14.57691434
Iteration 14, loss = 14.18594985
Iteration 15, loss = 14.07351495
Iteration 16, loss = 13.83342417
Iteration 17, loss = 13.63520827
Iteration 18, loss = 13.51883534
Iteration 19, loss = 13.34100600
Iteration 20, loss = 13.23170248
Iteration 21, loss = 13.07381911
Iteration 22, loss = 13.07461095
Iteration 23, loss = 12.87031023
Iteration 24, loss = 12.87552319
Iteration 25, loss = 12.69474057
Iteration 26, loss = 12.64445513
Iteration 27, loss = 12.53844473
Iteration 28, loss = 12.51107729
Iteration 29, loss = 12.33803976
Iteration 30, loss = 12.38699039
Iteration 31, loss

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (220) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 1, loss = 187.44265357
Iteration 2, loss = 81.73438444
Iteration 3, loss = 78.42157152
Iteration 4, loss = 76.85643834
Iteration 5, loss = 75.25989384
Iteration 6, loss = 73.69705720
Iteration 7, loss = 71.90571921
Iteration 8, loss = 70.76907469
Iteration 9, loss = 69.48328762
Iteration 10, loss = 68.07207723
Iteration 11, loss = 66.62871432
Iteration 12, loss = 65.37326032
Iteration 13, loss = 64.25574597
Iteration 14, loss = 63.68390289
Iteration 15, loss = 63.00569433
Iteration 16, loss = 62.69226966
Iteration 17, loss = 62.41794119
Iteration 18, loss = 61.81938801
Iteration 19, loss = 61.73003857
Iteration 20, loss = 61.33746453
Iteration 21, loss = 61.03127562
Iteration 22, loss = 60.76024923
Iteration 23, loss = 60.58565126
Iteration 24, loss = 60.16407146
Iteration 25, loss = 60.03999883
Iteration 26, loss = 59.76989818
Iteration 27, loss = 59.24561315
Iteration 28, loss = 59.12402103
Iteration 29, loss = 58.67237892
Iteration 30, loss = 58.47544273
Iteration 31, loss

MLPRegressor(max_iter=220, verbose=True)

In [20]:
mlp_reg_CA_2 = MLPRegressor(verbose=True)

mlp_reg_CA_2.fit(train[pred_vars], train['CA'])

mlp_reg_PA_2 = MLPRegressor(verbose=True)

mlp_reg_PA_2.fit(train[pred_vars], train['PA'])

Iteration 1, loss = 87.91469465
Iteration 2, loss = 24.19483408
Iteration 3, loss = 21.84100914
Iteration 4, loss = 21.53788065
Iteration 5, loss = 21.55672428
Iteration 6, loss = 21.40374112
Iteration 7, loss = 21.06303263
Iteration 8, loss = 20.36745872
Iteration 9, loss = 19.63264621
Iteration 10, loss = 19.02000869
Iteration 11, loss = 18.41923407
Iteration 12, loss = 17.39210603
Iteration 13, loss = 16.72220931
Iteration 14, loss = 16.43117692
Iteration 15, loss = 16.16598982
Iteration 16, loss = 15.85119795
Iteration 17, loss = 15.37816918
Iteration 18, loss = 15.07252736
Iteration 19, loss = 14.63937507
Iteration 20, loss = 14.52699626
Iteration 21, loss = 14.26658519
Iteration 22, loss = 14.07812844
Iteration 23, loss = 14.00529154
Iteration 24, loss = 13.79321449
Iteration 25, loss = 13.76673570
Iteration 26, loss = 13.51141554
Iteration 27, loss = 13.39879295
Iteration 28, loss = 13.30326322
Iteration 29, loss = 13.18493081
Iteration 30, loss = 13.07288238
Iteration 31, loss 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 1, loss = 149.92266909
Iteration 2, loss = 79.94704208
Iteration 3, loss = 75.57952494
Iteration 4, loss = 72.50512138
Iteration 5, loss = 70.24028677
Iteration 6, loss = 68.35529996
Iteration 7, loss = 66.55481933
Iteration 8, loss = 65.08629044
Iteration 9, loss = 63.76487370
Iteration 10, loss = 62.52998509
Iteration 11, loss = 61.91797077
Iteration 12, loss = 61.26035045
Iteration 13, loss = 60.57737578
Iteration 14, loss = 60.07515758
Iteration 15, loss = 60.09359274
Iteration 16, loss = 59.36146464
Iteration 17, loss = 59.14111906
Iteration 18, loss = 59.03544518
Iteration 19, loss = 58.81131983
Iteration 20, loss = 58.65267765
Iteration 21, loss = 58.68165069
Iteration 22, loss = 58.16249652
Iteration 23, loss = 58.22283654
Iteration 24, loss = 57.93816590
Iteration 25, loss = 57.62175704
Iteration 26, loss = 57.78910235
Iteration 27, loss = 57.49872653
Iteration 28, loss = 57.51825746
Iteration 29, loss = 57.21874630
Iteration 30, loss = 57.36376242
Iteration 31, loss

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(verbose=True)

In [ ]:
mlp_reg_CA_3 = MLPRegressor(max_iter=1000, verbose=True)

mlp_reg_CA_3.fit(train[pred_vars], train['CA'])

mlp_reg_PA_3 = MLPRegressor(max_iter=3000, verbose=True)

mlp_reg_PA_3.fit(train[pred_vars], train['PA'])

Iteration 1, loss = 102.74465741
Iteration 2, loss = 24.73574005
Iteration 3, loss = 22.16301355
Iteration 4, loss = 21.68832715
Iteration 5, loss = 21.53311866
Iteration 6, loss = 21.48871843
Iteration 7, loss = 21.04107211
Iteration 8, loss = 20.68806096
Iteration 9, loss = 20.12483145
Iteration 10, loss = 19.63072517
Iteration 11, loss = 18.96616233
Iteration 12, loss = 18.35930453
Iteration 13, loss = 17.95750323
Iteration 14, loss = 17.59074493
Iteration 15, loss = 17.40271749
Iteration 16, loss = 17.03983580
Iteration 17, loss = 16.88845890
Iteration 18, loss = 16.67755186
Iteration 19, loss = 16.55559576
Iteration 20, loss = 16.31798566
Iteration 21, loss = 15.99878131
Iteration 22, loss = 15.62697630
Iteration 23, loss = 15.24571727
Iteration 24, loss = 14.58630200
Iteration 25, loss = 14.09887560
Iteration 26, loss = 13.86743405
Iteration 27, loss = 13.74462014
Iteration 28, loss = 13.55308158
Iteration 29, loss = 13.37204428
Iteration 30, loss = 13.18022060
Iteration 31, loss

In [ ]:
#shuffle= False is a slight improvement for PA but takes a long time
mlp_reg_CA_4 = MLPRegressor(max_iter=1000, verbose=True, n_iter_no_change=25)

mlp_reg_CA_4.fit(train[pred_vars], train['CA'])
print('First Trained!')
mlp_reg_PA_4 = MLPRegressor(max_iter=3000, verbose=True, n_iter_no_change=25)

mlp_reg_PA_4.fit(train[pred_vars], train['PA'])

In [ ]:
#Shuffle and n_iter_no_change cause it to never converge, most likely it is learning the pre shuffled dataframe
'''
mlp_reg_CA_5 = MLPRegressor(max_iter=1000, verbose=True, n_iter_no_change=25, shuffle= False)

mlp_reg_CA_5.fit(train[pred_vars], train['CA'])
print('First Trained!')
mlp_reg_PA_5 = MLPRegressor(max_iter=1000, verbose=True, n_iter_no_change=25, shuffle= False)

mlp_reg_PA_5.fit(train[pred_vars], train['PA'])
'''

In [ ]:
#Second best so farto Random Forest
from sklearn.neighbors import KNeighborsClassifier

kn_reg_PA = KNeighborsClassifier()
kn_reg_PA.fit(train[pred_vars], train['PA'])

kn_reg_CA = KNeighborsClassifier()
kn_reg_CA.fit(train[pred_vars], train['CA'])

In [ ]:
reg_multi_PA = sm.OLS(train['PA'], train[pred_vars], hasconst=False).fit()
reg_multi_PA.summary()

In [ ]:
reg_multi_CA = sm.OLS(train['CA'], train[pred_vars], hasconst=False).fit()
reg_multi_CA.summary()

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

knr_reg_PA = KNeighborsRegressor()
knr_reg_PA.fit(train[pred_vars], train['PA'])

knr_reg_CA = KNeighborsRegressor()
knr_reg_CA.fit(train[pred_vars], train['CA'])

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dt_reg_PA = DecisionTreeRegressor()

dt_reg_PA.fit(train[pred_vars], train['PA'])

dt_reg_CA = DecisionTreeRegressor()

dt_reg_CA.fit(train[pred_vars], train['CA'])

In [ ]:
#RMSE PA
#Starts 7:33 7:44 15 mins
modelsPA = [rf_reg_PA, mlp_reg_PA, mlp_reg_PA_2, mlp_reg_PA_3, mlp_reg_PA_4, mlp_reg_PA_5, kn_reg_PA, 
            reg_multi_PA, knr_reg_PA, dt_reg_PA] #svr_reg_PA]

for reg in modelsPA:
    reg_pred = reg.predict(test[pred_vars])

    reg_rmse = metrics.mean_squared_error(test['PA'], reg_pred, squared=False)
    print(reg, "RMSE:", reg_rmse)

In [ ]:
#RMSE CA
modelsCA = [rf_reg_CA, mlp_reg_CA, mlp_reg_CA_2, mlp_reg_CA_3, mlp_reg_CA_4, mlp_reg_CA_5, kn_reg_CA, 
            reg_multi_CA, knr_reg_CA, dt_reg_CA] #svr_reg_CA]
for reg in modelsCA:
    
    reg_pred = reg.predict(test[pred_vars])

    reg_rmse = metrics.mean_squared_error(test['CA'], reg_pred, squared=False)
    print(reg, "RMSE:", reg_rmse)

In [271]:
#R squared PA train
#Starts 7:33 7:44 15 mins
modelsPA = [rf_reg_PA, mlp_reg_PA, mlp_reg_PA_2, mlp_reg_PA_3, mlp_reg_PA_4, mlp_reg_PA_5, kn_reg_PA, 
            knr_reg_PA, dt_reg_PA, svr_reg_PA]

for reg in modelsPA:

    reg_r = reg.score(train[pred_vars], train['PA'])
    print(reg, "R squared:", reg_r)

RandomForestRegressor() R squared: 0.9690367884519367
MLPRegressor(max_iter=220) R squared: 0.8090695156029543
MLPRegressor() R squared: 0.8053661600812837
MLPRegressor(max_iter=3000) R squared: 0.809758337399105
MLPRegressor(max_iter=3000, n_iter_no_change=25, verbose=True) R squared: 0.812437133499198
MLPRegressor(max_iter=1000, n_iter_no_change=25, shuffle=False, verbose=True) R squared: 0.8147664661300763
KNeighborsClassifier() R squared: 0.250391481365487
KNeighborsRegressor() R squared: 0.8013301905932495
DecisionTreeRegressor() R squared: 1.0
SVR() R squared: 0.7119027672514842


In [281]:
#R squared CA train
modelsCA = [rf_reg_CA, mlp_reg_CA, mlp_reg_CA_2, mlp_reg_CA_3, mlp_reg_CA_4, mlp_reg_CA_5, kn_reg_CA, 
            knr_reg_CA, dt_reg_CA, svr_reg_CA]
for reg in modelsCA:
    
    reg_r = reg.score(train[pred_vars], train['CA'])
    print(reg, "R squared:", reg_r)

RandomForestRegressor() R squared: 0.9899100895579698
MLPRegressor(max_iter=220) R squared: 0.9559677928827406
MLPRegressor() R squared: 0.9581138402842833
MLPRegressor(max_iter=1000) R squared: 0.9559072623444695
MLPRegressor(max_iter=1000, n_iter_no_change=25, verbose=True) R squared: 0.9627970818981004
MLPRegressor(max_iter=1000, n_iter_no_change=25, shuffle=False, verbose=True) R squared: 0.9619423138949014
KNeighborsClassifier() R squared: 0.2693852360862917
KNeighborsRegressor() R squared: 0.9182313809774291
DecisionTreeRegressor() R squared: 1.0
SVR() R squared: 0.9110607911965478


In [277]:
#R squared PA test
#Starts 7:33 7:44 15 mins
import warnings
warnings.filterwarnings("ignore")

modelsPA = [rf_reg_PA, mlp_reg_PA, mlp_reg_PA_2, mlp_reg_PA_3, mlp_reg_PA_4, mlp_reg_PA_5, kn_reg_PA, 
            reg_multi_PA, knr_reg_PA, dt_reg_PA, svr_reg_PA]

for reg in modelsPA:
    test['PA_pred'] = reg.predict(test[pred_vars])
    test['PA_diff'] = test['PA_pred'] - test['PA']
    test['PA_diff_abs'] = test['PA_diff'].abs()
    
    print(reg, 'Avg Difference', test['PA_diff'].sum()/36188)
    print(reg, 'Avg Difference absolute value', test['PA_diff_abs'].sum()/36188)

RandomForestRegressor() Avg Difference 0.030276334696584518
RandomForestRegressor() Avg Difference absolute value 8.91360672045982
MLPRegressor(max_iter=220) Avg Difference -0.2067783086984525
MLPRegressor(max_iter=220) Avg Difference absolute value 8.515157593748608
MLPRegressor() Avg Difference 1.22326001866441
MLPRegressor() Avg Difference absolute value 8.645912451225966
MLPRegressor(max_iter=3000) Avg Difference -0.5013251638702374
MLPRegressor(max_iter=3000) Avg Difference absolute value 8.501565804745429
MLPRegressor(max_iter=3000, n_iter_no_change=25, verbose=True) Avg Difference -0.10113137492177063
MLPRegressor(max_iter=3000, n_iter_no_change=25, verbose=True) Avg Difference absolute value 8.535128396667307
MLPRegressor(max_iter=1000, n_iter_no_change=25, shuffle=False, verbose=True) Avg Difference -0.46994744179320525
MLPRegressor(max_iter=1000, n_iter_no_change=25, shuffle=False, verbose=True) Avg Difference absolute value 8.514596664818411
KNeighborsClassifier() Avg Differ

In [279]:
#R squared CA test
modelsCA = [rf_reg_CA, mlp_reg_CA, mlp_reg_CA_2, mlp_reg_CA_3, mlp_reg_CA_4, mlp_reg_CA_5, kn_reg_CA, 
            reg_multi_CA, knr_reg_CA, dt_reg_CA, svr_reg_CA]

warnings.filterwarnings("ignore")
for reg in modelsCA:
    test['CA_pred'] = reg.predict(test[pred_vars])
    test['CA_diff'] = test['CA_pred'] - test['CA']
    test['CA_diff_abs'] = test['CA_diff'].abs()
    
    print(reg, 'Avg Difference', test['CA_diff'].sum()/36188)
    print(reg, 'Avg Difference absolute value', test['CA_diff_abs'].sum()/36188)

RandomForestRegressor() Avg Difference -0.18437133856527027
RandomForestRegressor() Avg Difference absolute value 4.967288327622416
MLPRegressor(max_iter=220) Avg Difference 0.8804179155167604
MLPRegressor(max_iter=220) Avg Difference absolute value 4.095235959958195
MLPRegressor() Avg Difference 0.278850731323083
MLPRegressor() Avg Difference absolute value 4.009609168467823
MLPRegressor(max_iter=1000) Avg Difference 0.7938086143408916
MLPRegressor(max_iter=1000) Avg Difference absolute value 4.071032601286901
MLPRegressor(max_iter=1000, n_iter_no_change=25, verbose=True) Avg Difference -0.2812159805828502
MLPRegressor(max_iter=1000, n_iter_no_change=25, verbose=True) Avg Difference absolute value 3.7817995620423153
MLPRegressor(max_iter=1000, n_iter_no_change=25, shuffle=False, verbose=True) Avg Difference -0.8498997186218273
MLPRegressor(max_iter=1000, n_iter_no_change=25, shuffle=False, verbose=True) Avg Difference absolute value 3.8217689645499515
KNeighborsClassifier() Avg Differ

In [283]:
#MSE CA
modelsCA = [rf_reg_CA, mlp_reg_CA, mlp_reg_CA_2, mlp_reg_CA_3, mlp_reg_CA_4, mlp_reg_CA_5, kn_reg_CA, 
            reg_multi_CA, knr_reg_CA, dt_reg_CA, svr_reg_CA]
for reg in modelsCA:
    
    reg_pred = reg.predict(test[pred_vars])

    reg_mse = metrics.mean_squared_error(test['CA'], reg_pred, squared=True)
    print(reg, "MSE:", reg_mse)

RandomForestRegressor() MSE: 44.82212115894772
MLPRegressor(max_iter=220) MSE: 29.369693882142183
MLPRegressor() MSE: 28.28871930543615
MLPRegressor(max_iter=1000) MSE: 29.121424069616598
MLPRegressor(max_iter=1000, n_iter_no_change=25, verbose=True) MSE: 25.664985325960956
MLPRegressor(max_iter=1000, n_iter_no_change=25, shuffle=False, verbose=True) MSE: 26.257158588925556
KNeighborsClassifier() MSE: 205.9765944511993
<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000182C9FA6DC0> MSE: 56.803341222901004
KNeighborsRegressor() MSE: 76.46874765115508
DecisionTreeRegressor() MSE: 119.7895711285509
SVR() MSE: 54.868037628925805


In [282]:
#MSE PA

modelsPA = [rf_reg_PA, mlp_reg_PA, mlp_reg_PA_2, mlp_reg_PA_3, mlp_reg_PA_4, mlp_reg_PA_5, kn_reg_PA, 
            reg_multi_PA, knr_reg_PA, dt_reg_PA, svr_reg_PA]

for reg in modelsPA:
    reg_pred = reg.predict(test[pred_vars])

    reg_mse = metrics.mean_squared_error(test['PA'], reg_pred, squared=True)
    print(reg, "MSE:", reg_mse)

RandomForestRegressor() MSE: 136.72060260859953
MLPRegressor(max_iter=220) MSE: 124.69673378937641
MLPRegressor() MSE: 126.88523427770754
MLPRegressor(max_iter=3000) MSE: 124.5214031306136
MLPRegressor(max_iter=3000, n_iter_no_change=25, verbose=True) MSE: 124.66650649557323
MLPRegressor(max_iter=1000, n_iter_no_change=25, shuffle=False, verbose=True) MSE: 124.60039114166439
KNeighborsClassifier() MSE: 431.86675140930697
<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000182C417B9A0> MSE: 173.4784952216835
KNeighborsRegressor() MSE: 185.75879849673927
DecisionTreeRegressor() MSE: 309.39515861611585
SVR() MSE: 177.1463787744576


# Adjustments to models

In [61]:
new_CA_pred_vars = corr2[corr2['CA'] >= .25].index.tolist()

In [62]:
new_PA_pred_vars = corr2[corr2['PA'] >= .25].index.tolist()

In [63]:
new_CA_pred_vars.remove('CA')
new_CA_pred_vars.remove('PA')
new_PA_pred_vars.remove('CA')
new_PA_pred_vars.remove('PA')

Test models again